In [78]:
%%capture
import sys
import numpy as np

%pip install pyomo >/dev/null 2>/dev/null
%pip install highspy >/dev/null 2>/dev/null

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

import pandas as pd
import csv


In [79]:
#Initialize the model
model = pyo.ConcreteModel("National Park Trip")

In [80]:
#Importing the data
#activity_table = pd.read_csv('/Users/nikitamehendale/Downloads/ieor164-mini-project/164 Mini Project Data  - Activities.csv')
#activity_table = pd.read_csv('164 Mini Project Data  - Activities.csv')
activity_table = pd.read_excel("164 Mini Project Data .xlsx", sheet_name="Activities")
activity_table.head()
#network flow table with travel times/cost
#list of Lodging Activities

,Activity,Name,Cost,Duration,Fun Points,Category,Park,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,0,Congress Trail - Sequoia,0.0,3.0,NaN,Hiking,Sequoia,NaN,NaN,NaN
1,1,Crescent Meadown Loop - Sequoia,0.0,1.0,NaN,Hiking,Sequoia,NaN,NaN,NaN
2,2,Tokopah Falls - Sequoia,0.0,3.0,NaN,Hiking,Sequoia,NaN,NaN,NaN
3,3,Sunset at Moro Rock - Sequoia,0.0,3.0,NaN,Hiking,Sequoia,NaN,NaN,NaN
4,4,Tunnel Log - Sequoia,0.0,1.0,2.0,Driving,Sequoia,NaN,NaN,NaN


In [81]:
# Sets
#activity_table = activity_table.set_index("Activity")
model.I = pyo.Set(initialize=activity_table['Activity'].dropna().tolist()) #set of activities
#model.P = pyo.Set(initialize=['Hiking', 'Adventures', 'Shopping', 'Driving', 'Vistas']) # category of activities
model.K = pyo.Set(initialize=[1,2,3]) #set of days
model.L = pyo.Set() #set of lodging options from its own file
model.N = pyo.Set() #set of all nodes in network flow (lodging + activities)

In [82]:
#Parameters
duration_dict = activity_table.set_index("Activity")["Duration"].to_dict() #duration of each activity
activityType_dict = activity_table.set_index("Activity")["Category"].to_dict() #type of each activity
cost_dict = activity_table.set_index("Activity")["Cost"].to_dict() #cost of each activity
funPoints_dict = activity_table.set_index("Activity")["Fun Points"].to_dict() #fun points of each activity

model.duration = pyo.Param(model.I, initialize=duration_dict)
model.activity_type = pyo.Param(model.I, initialize=activityType_dict, within=pyo.Any)
model.cost = pyo.Param(model.I, initialize=cost_dict)
model.fun_points = pyo.Param(model.I, initialize=funPoints_dict)



In [83]:
#Variables
model.x = pyo.Var(model.I, model.K, within=pyo.Binary) #activity selection 
model.y = pyo.Var(model.N, model.N, within=pyo.Binary) #path between locations (determines order)
model.h = pyo.Var(model.L, model.K, within=pyo.Binary) #lodging selection

In [91]:
#Constraints
def hiking_time(model, k):
    return sum(model.duration[i] * model.x[i, k] for i in model.I if model.activity_type[i] == 'Hiking') <= 5
model.hiking_time = pyo.Constraint(model.K, rule=hiking_time)

def adventure_budget(model, k):
    return sum(model.cost[i] * model.x[i, k] for i in model.I if model.activity_type[i] == 'Adventures') <= 600
model.adventure_budget = pyo.Constraint(model.K, rule=adventure_budget)

def min_fun(model):
    return sum(model.fun_points[i] * model.x[i, k] for i in model.I for k in model.K) >= 5
model.min_fun = pyo.Constraint(rule=min_fun)   

def min_activities(model):
    return sum(model.x[i, k] for i in model.I for k in model.K) >= 1
model.min_activities = pyo.Constraint(rule=min_activities)


'''
def scheduling_continutity(model, i):
    return sum(model.y[i,j] for j in model.N) == sum(model.x[i,p,k] for p in model.P for k in model.K)
model.scheduling_continuity = pyo.Constraint(model.I, rule=scheduling_continutity)
'''
'''
def travel_time(model):
    return sum(model.y[i,j] * r_ij[i,j] for i,j in model.N) <= sum(model.x[i,p,k] * t_ip[i] for i in model.I for p in model.P for k in model.K)
model.travel_time = pyo.Constraint(model.K, rule=travel_time)
'''

'pyomo.core.base.constraint.IndexedConstraint'>) on block National Park Trip
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
National Park Trip with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.constraint.ScalarConstraint'>) on block National Park Trip
with a new Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 'pyomo.core.base.constraint.ScalarConstraint'>) on block National
Park Trip with a new Component (type=<class


'\ndef travel_time(model):\n    return sum(model.y[i,j] * r_ij[i,j] for i,j in model.N) <= sum(model.x[i,p,k] * t_ip[i] for i in model.I for p in model.P for k in model.K)\nmodel.travel_time = pyo.Constraint(model.K, rule=travel_time)\n'

In [92]:
#Objective Function
def total_cost(model):
    #lodging_cost = sum(model.h[l, k] * activity_table.loc[l, 'Cost'] for l in model.L for k in model.K)
    activity_cost = sum(model.x[i,k] * activity_table.at[i, 'Cost'] for i in model.I for k in model.K)
    #car_cost = sum(model.y[i,j] * d_ij[i,j] for i in model.N for j in model.N) # need to re-do this constraint to reference the proper distance param from data tables
    return activity_cost

model.obj = pyo.Objective(rule=total_cost, sense=pyo.minimize)

'pyomo.core.base.objective.ScalarObjective'>) on block National Park Trip with
a new Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>).
This is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [93]:
#Solve the model
SOLVER.solve(model)
model.display()

Model National Park Trip

  Variables:
    x : Size=87, Index=I*K
        Key     : Lower : Value : Upper : Fixed : Stale : Domain
         (0, 1) :     0 :   0.0 :     1 : False : False : Binary
         (0, 2) :     0 :   0.0 :     1 : False : False : Binary
         (0, 3) :     0 :   0.0 :     1 : False : False : Binary
         (1, 1) :     0 :   0.0 :     1 : False : False : Binary
         (1, 2) :     0 :   0.0 :     1 : False : False : Binary
         (1, 3) :     0 :   0.0 :     1 : False : False : Binary
         (2, 1) :     0 :   0.0 :     1 : False : False : Binary
         (2, 2) :     0 :   0.0 :     1 : False : False : Binary
         (2, 3) :     0 :   0.0 :     1 : False : False : Binary
         (3, 1) :     0 :   0.0 :     1 : False : False : Binary
         (3, 2) :     0 :   0.0 :     1 : False : False : Binary
         (3, 3) :     0 :   0.0 :     1 : False : False : Binary
         (4, 1) :     0 :   1.0 :     1 : False : False : Binary
         (4, 2) :     0 